In [85]:
import tensorflow as tf
import pandas as pd
import numpy as np
import cv2
from tqdm import tqdm
import sklearn as sk
import os
import random
import math
import datetime


In [86]:
tf.test.is_built_with_cuda()

True

# Data Pipeline

Extract video frames

In [87]:
IMG_SIZE = 224

v1 = "video_data/violence_dataset/Violence"
v2 = "video_data/surveillance_dataset/fight"
n1 = "video_data/violence_dataset/NonViolence"
n2 = "video_data/surveillance_dataset/nofight"
violent_videos = [v1, v2]
nonviolent_videos = [n1, n2]
DATA_PATHS = [violent_videos, nonviolent_videos]

NEW_FOLDER = "new_data_dir"

''' make the list of paths to the videos
    extract the frames from the videos into images (what if i just extract two frames from each?)
    save these images in a new folder
    this folder is now your data dir
    make sure that each frame is in its respective folder (violence, non violence)'''

we have video_to_frames(path) function, what I need to do is use [violent_videos, nonviolent_videos] to automatically go into NEW_FOLDER accordingly their new folders with the frames extracted already

so change video_to_frames(list) to take in a list of lists or strings

In [88]:
def video_to_frames(p) -> list:
    
    #assert os.path.exists(p)
    cap = cv2.VideoCapture(p)
    
    #assert cap
    rate = math.floor(cap.get(3))

    ImageFrames = []
    #assert cap.isOpened()
    

    '''make sure to figure out how to extract a certain amount of frames so you don't use too much memory'''
    
    while cap.isOpened():
        #ID = cap.get(1)
        
        success, image = cap.read()
        n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        mid = int(n_frames/2)
        first_q = int(n_frames/4)
        third_q = int(n_frames * .75)
        end = n_frames
        #print(n_frames)
        #assert success

        if success:

            for frame in range(n_frames):
                if frame == first_q or frame == mid or frame == third_q or frame == end:
                    
                    
                    image = cv2.resize(image, (IMG_SIZE, IMG_SIZE))
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                    ImageFrames.append(image)

                    cv2.imwrite()
                else: continue
            cap.release()
    #print("successful frame capture")
            return ImageFrames

        else: 
            break

In [89]:
CATEGORIES = ["NonViolence", "Violence"]
X_set = []
y_set = []
# this is for using the video_to_frames to extract the frames into a list
for category in os.listdir(PATH):
    
    
    path = os.path.join(PATH, category)
    #print(path)
    #assert os.path.exists(path)


    class_num = CATEGORIES.index(category)
    for i, video in tqdm(enumerate(os.listdir(path)), total=len(os.listdir(path))):

        #print(path + '/' + video)
        #print(os.path.join(path, video))
        frames = video_to_frames(path + '/' + video)
        if frames:
        
            for j, frame in enumerate(frames):
                X_set.append(frame)
                y_set.append(class_num)
        else: continue

            
print("success")

100%|██████████| 150/150 [00:02<00:00, 58.72it/s]

success


In [90]:
X_set = np.array(X_set).reshape(-1 , IMG_SIZE * IMG_SIZE * 3)
y_set = np.array(y_set)
len(X_set), len(y_set)

(900, 900)

In [91]:
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split

stratified_sample = StratifiedShuffleSplit(n_splits=2, test_size=0.2)

for train_index, test_index in stratified_sample.split(X_set, y_set):
    X_train, X_test = X_set[train_index], X_set[test_index]
    y_train, y_test = y_set[train_index], y_set[test_index]

In [92]:
X_train = X_train.reshape(-1, IMG_SIZE, IMG_SIZE, 3) / 255
X_test_nn = X_test.reshape(-1, IMG_SIZE, IMG_SIZE, 3) / 255
#X_val = X_val.reshape(-1, IMG_SIZE, IMG_SIZE, 3) / 255

y_train = np.asarray(y_train)
y_test_nn = np.asarray(y_test)
#y_val = np.asarray(y_val)

In [93]:
from tensorflow.keras.utils import Sequence


class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

train_gen = DataGenerator(X_train, y_train, 4)
test_gen = DataGenerator(X_test_nn, y_test_nn, 4)
#val_gen = DataGenerator(X_val, y_val, 16)

In [94]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.efficientnet import preprocess_input

In [95]:
from tensorflow.keras import Sequential, layers


data_augmentation = Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation"
)

## Building the Model

In [102]:
from tensorflow.keras.applications.efficientnet_v2 import EfficientNetV2B0
from tensorflow.keras.applications.mobilenet import MobileNet 
class OwnEfficientNetB0:
    @staticmethod
    def build(input_shape, data_augmentation, trainable=False, dropout=0.7):
        inputs = keras.Input(shape=input_shape)
        x = data_augmentation(inputs)
        x = preprocess_input(x)

        baseModel = MobileNet(weights="imagenet", include_top=False, input_tensor=x)
        baseModel.trainable = trainable

        headModel = baseModel.output
        headModel = layers.GlobalAveragePooling2D()(headModel)
        headModel = layers.Dropout(dropout)(headModel)
        outputs = layers.Dense(1, activation="sigmoid")(headModel)
        model = Model(inputs, outputs)

        return model

In [103]:
model = OwnEfficientNetB0.build(input_shape=(IMG_SIZE,IMG_SIZE) + (3,), data_augmentation=data_augmentation)

In [104]:
model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_8 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 img_augmentation (Sequentia  (None, 224, 224, 3)      0         
 l)                                                              
                                                                 
 conv1 (Conv2D)              (None, 112, 112, 32)      864       
                                                                 
 conv1_bn (BatchNormalizatio  (None, 112, 112, 32)     128       
 n)                                                              
                                                                 
 conv1_relu (ReLU)           (None, 112, 112, 32)      0         
                                                                 
 conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)     288 

Compile the model

In [105]:
#keep batchlayers intact, reduces learning time
for layer in model.layers:
    if isinstance(layer, layers.BatchNormalization):
        layer.trainable = False
    else:
        layer.trainable = True
model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
            loss=tf.keras.losses.BinaryCrossentropy(),
            metrics=keras.metrics.BinaryAccuracy())

Fit the model

In [106]:
from keras.callbacks import EarlyStopping
callbacks = [
    EarlyStopping(monitor='val_loss', patience=30, mode='min', min_delta=0.0001)]

In [107]:
history = model.fit(
    train_gen, epochs=25, callbacks=callbacks
)


Epoch 1/25
180/180 [==============================] - 10s 41ms/step - loss: 2.2766 - binary_accuracy: 0.4833
Epoch 2/25
180/180 [==============================] - 7s 41ms/step - loss: 1.5950 - binary_accuracy: 0.4667
Epoch 3/25
 55/180 [========>.....................] - ETA: 5s - loss: 1.2728 - binary_accuracy: 0.4909

KeyboardInterrupt: 

# Fine Tuning